In [1]:
# https://towardsdatascience.com/snorkel-programmatically-build-training-data-in-python-712fc39649fe

In [2]:
!pip install snorkel

# Load Data

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_pickle(
    "./data/train_fake_jobs.pkl"
)
test_df = pd.read_pickle(
    "./data/test_fake_jobs.pkl"
)

# Create Labeling Functions

In [5]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis

In [6]:
FAKE = 1
REAL = 0
ABSTAIN = -1

In [7]:
FAKE_COMPANIES = [
    "Aker Solutions",
    "Aptitude Staffing Solutions",
    "Gary Cartwright",
    "Edison International and Refined Resources",
    "Le Meridien",
]

In [8]:
@labeling_function()
def no_requirements(x: pd.Series):

    return FAKE if x.requirements == "" else ABSTAIN


@labeling_function()
def requirements_less_than_10(x: pd.Series):
    num_words = len(x.requirements.split(" "))
    return FAKE if num_words <= 10 else ABSTAIN


@labeling_function()
def requirements_less_than_20(x: pd.Series):
    num_words = len(x.requirements.split(" "))
    return FAKE if num_words <= 20 else ABSTAIN


@labeling_function()
def no_company_profile(x: pd.Series):
    return FAKE if x.company_profile == "" else ABSTAIN


@labeling_function()
def no_company_logo(x: pd.Series):
    return FAKE if x.has_company_logo == 0 else ABSTAIN


@labeling_function()
def suspicious_company(x: pd.Series):
    return (
        FAKE
        if any(
            company.lower() in x.company_profile.lower() for company in FAKE_COMPANIES
        )
        else ABSTAIN
    )


@labeling_function()
def has_background_check(x: pd.Series):
    return REAL if "background check" in x.requirements else ABSTAIN


@labeling_function()
def required_experience(x: pd.Series):
    return REAL if pd.notna(x.required_experience) else ABSTAIN


@labeling_function()
def required_education(x: pd.Series):
    return REAL if pd.notna(x.required_education) else ABSTAIN

# Apply Labeling Functions to the Data

In [9]:
lfs = [
    no_company_profile,
    suspicious_company,
    no_company_logo,
    has_background_check,
    required_experience,
    required_education,
]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train_df)

100%|██████████| 13410/13410 [00:01<00:00, 6911.58it/s]


# Evaluate Labeling Functions

In [10]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary(Y=train_df.fraudulent.values)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
no_company_profile,0,[1],0.186204,0.170694,0.110365,459,2038,0.183821
suspicious_company,1,[1],0.006861,0.005667,0.005667,92,0,1.000000
no_company_logo,2,[1],0.205742,0.161894,0.101566,459,2300,0.166365
has_background_check,3,[0],0.035496,0.032438,0.002759,464,12,0.974790
required_experience,4,[0],0.607010,0.507755,0.119165,7809,331,0.959337
required_education,5,[0],0.551603,0.489485,0.101790,7075,322,0.956469


## Evaluate Conflicts

In [11]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 2], L_train[:, 4])

res = train_df.iloc[buckets[(FAKE, REAL)]].sample(10, random_state=1)[
    ["has_company_logo", "required_experience", "fraudulent"]
]

res

,has_company_logo,required_experience,fraudulent
16877,0,Mid-Senior level,0
17068,0,Mid-Senior level,0
16816,0,Mid-Senior level,0
12186,0,Not Applicable,0
16808,0,Mid-Senior level,0
678,0,Not Applicable,0
3706,0,Entry level,0
14150,0,Entry level,0
2199,0,Entry level,0
4411,0,Entry level,0


# Combine Predictions

## MajorityLabelVoter

In [12]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

## LabelModel

In [13]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=1)


INFO:root:Computing O...
INFO:root:Estimating \mu...
 38%|███▊      | 192/500 [00:00<00:00, 969.60epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.011]
INFO:root:[300 epochs]: TRAIN:[loss=0.010]
100%|██████████| 500/500 [00:00<00:00, 1014.53epoch/s]
INFO:root:Finished Training


In [14]:
L_test = applier.apply(df=test_df)

100%|██████████| 4470/4470 [00:00<00:00, 7315.75it/s]


In [15]:
Y_train = train_df["fraudulent"]
Y_test = test_df["fraudulent"]

In [16]:
majority_acc = majority_model.score(
    L=L_test,
    Y=Y_test,
)["accuracy"]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

Majority Vote Accuracy:   80.8%


In [17]:
label_model_acc = label_model.score(L=L_test, Y=Y_test)["accuracy"]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Label Model Accuracy:     71.9%
